# Introduction to the Reverse Osmosis Unit Model
This tutorial shows how to build, initialize, simulate, and optimize a reverse osmosis (RO) unit model using WaterTAP. In this example, we will minimize the specific energy consumption of the RO unit.

In [1]:
# Python magics from dsl.py
%load_ext dsl
# Utility functions to retrieve result objects
from dsl import get_model, get_results

In [2]:
%%iic
clear
unit ro-0D

Action: clear model
Action: unit RO-0D


In [5]:
%ii set inlet flow mass ?

ValueError: Cannot find port flow for unit

In [4]:
%%iic
clear
# create unit
unit RO-0D
# fix values
set inlet flow mass NaCl 0.035
set inlet flow mass H2O 0.965
set inlet pressure 50e5
set inlet temperature 298.15
# set membrane area 50
# set membrane permeability water 4.2e-12
# set membrane permeability salt 3.5e-8
# set permeate pressure 101325
# scale
scale flow mass NaCl 1e2
scale flow mass H2O 1
# solve
init
solve

Action: clear model
Comment: create unit
Action: unit RO-0D
Comment: fix values
Action: fix inlet flow mass NaCl <- 0.035
Action: fix inlet flow mass H2O <- 0.965
Action: fix inlet pressure <- 5000000.0
Action: fix inlet temperature <- 298.15
Comment: set membrane area 50
Comment: set membrane permeability water 4.2e-12
Comment: set membrane permeability salt 3.5e-8
Comment: set permeate pressure 101325
Comment: scale
Unknown command: scale
Unknown command: scale
Comment: solve
Unknown command: init
Unknown command: solve


In [3]:
get_model().fs.unit.display()

AttributeError: 'NoneType' object has no attribute 'fs'

## Stopped here! Code below is standard IDAES Python

## Step 7: Unfix variables, set variable bounds, and run optimization to minimize specific energy consumption.

In [ ]:
# Unfix membrane area and feed pressure
m.fs.unit.area.unfix()                  # membrane area (m^2)
m.fs.unit.inlet.pressure[0].unfix()     # feed pressure (Pa)

In [ ]:
# Set lower and upper bounds for membrane area (m^2)
m.fs.unit.area.setlb(1)
m.fs.unit.area.setub(500)

In [ ]:
# Set lower and upper bounds for feed pressure (Pa)
m.fs.unit.inlet.pressure[0].setlb(10e5)
m.fs.unit.inlet.pressure[0].setub(80e5)

In [ ]:
# Assume 100% efficiency of pumps and ERD and no pressure losses
#--> Pump power consumption ~ Qp*Pf/3.6e6
m.fs.specific_energy_consumption = Expression(
    expr=m.fs.unit.inlet.pressure[0]/(3.6e6))

In [ ]:
# Define objective function to minimize the specific energy consumption.
m.fs.objective = Objective(expr=m.fs.specific_energy_consumption)

In [ ]:
# Set the water recovery to 50%
m.fs.unit.recovery_vol_phase[0,'Liq'].fix(0.50)

In [ ]:
# The solver will find the membrane area and 
# inlet pressure that achieve 50% recovery while minimizing
# specific energy consumption. Since we fixed the 
# volumetric water recovery, a degree of freedom 
# was removed from the model.
print(degrees_of_freedom(m))

In [ ]:
optimization_results = solver.solve(m)

In [ ]:
# membrane area of the optimized RO unit
value(m.fs.unit.area)

In [ ]:
# inlet pressure of the optimized RO unit
value(m.fs.unit.inlet.pressure[0])

In [ ]:
# the minimum specific energy consumption
value(m.fs.specific_energy_consumption)

In [ ]:
# display the overall report on the RO unit
m.fs.unit.report()

In [4]:
class T:
    def __getitem__(self, key):
        print(f"getitem, key={key}")

In [5]:
t = T()

In [6]:
t[1,23]

getitem, key=(1, 23)


In [7]:
x = [1,2,3]

In [8]:
t[x]

getitem, key=[1, 2, 3]


In [11]:
t[tuple(x)]

getitem, key=(1, 2, 3)
